In [42]:
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests


In [43]:
df = pd.read_excel('isbntest.xlsx',names=['nr','isbn'], dtype='string')
df = df.apply(lambda x: x.str.strip())
df

,nr,isbn
0,1,9783964450685
1,2,9783868215342
2,3,9783779965275
3,4,9783826076473
4,5,9783518299197
5,6,9783593509938
6,7,9783960420545
7,8,9783960420552
8,9,9783960420569
9,10,3937390227


In [44]:
base_url = 'https://services.dnb.de/sru/dnb?'
params = {'recordSchema' : 'MARC21plus-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
         }

In [45]:
def gnd_abfrage(row):
    params.update({'query': f'NUM={row.isbn}'})
    response = requests.get(base_url, params=params)
    response_xml = soup(response.content, features="xml")

    try:
        matches = list()
        for record in response_xml.find_all('record', {'type':'Bibliographic'}):
            record_match = dict()
            #idn bestimmen
            try:
                record_match['idn'] = record.find('controlfield', {'tag': '001'}).string.strip()
            except:
                record_match['idn'] = None


            try:
                record_match['tit'] = record.find('datafield', {'tag': '245'}).find('subfield', {'code':'a'}).string.strip()
            except:
                record_match['tit'] = None


            try:
                record_match['aut'] = record.find('datafield', {'tag': '100'}).find('subfield', {'code':'a'}).string.strip()
            except:
                record_match['aut'] = None

            try:
                record_match['jhr'] = record.find('datafield', {'tag': '264'}).find('subfield', {'code':'c'}).string.strip()
            except:
                record_match['jhr'] = None

            try:
                record_match['vlg'] = record.find('datafield', {'tag': '264'}).find('subfield', {'code':'b'}).string.strip()
            except:
                record_match['vlg'] = None



            matches.append(record_match)
        return matches
    except Exception as e:
        return "fehler %r" % e

In [46]:
df['idn'] = df.apply(gnd_abfrage, axis=1)
df

,nr,isbn,idn
0,1,9783964450685,"[{'idn': '1246969653', 'tit': 'Rude girl', 'au..."
1,2,9783868215342,"[{'idn': '1049849752', 'tit': 'Bilderbuch und ..."
2,3,9783779965275,"[{'idn': '1231247207', 'tit': 'Kritische sozia..."
3,4,9783826076473,"[{'idn': '1257538810', 'tit': 'Was ist Schönh..."
4,5,9783518299197,"[{'idn': '1197070176', 'tit': 'Down girl', 'au..."
5,6,9783593509938,"[{'idn': '1162456698', 'tit': 'Wie männlich i..."
6,7,9783960420545,"[{'idn': '1172148945', 'tit': 'Politiken der I..."
7,8,9783960420552,"[{'idn': '1172149658', 'tit': 'Subjekte der In..."
8,9,9783960420569,"[{'idn': '1172148759', 'tit': 'Theorien der In..."
9,10,3937390227,"[{'idn': '971153302', 'tit': 'Strategien der W..."


In [47]:
base_url = 'https://services.dnb.de/sru/dnb?'
params = {'recordSchema' : 'MARC21plus-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
         }

In [48]:
def gnd_abfragen(row):
    params.update({'query': f'NUM={row.isbn}'})
    response = requests.get(base_url, params=params)
    response_xml = soup(response.content, features="xml")

    try:
        matches = list()
        for record in response_xml.find_all('record', {'type':'Holdings'}):
            record_match = dict()       

            try:
                record_match['sig'] = record.find('datafield', {'tag': '852'}).find('subfield', {'code':'c'}).string.strip()
            except:
                record_match['sig'] = None

            matches.append(record_match)
        return matches
    except Exception as e:
        return "fehler %r" % e

In [49]:
df['exemplar'] = df.apply(gnd_abfragen, axis=1)
df

,nr,isbn,idn,exemplar
0,1,9783964450685,"[{'idn': '1246969653', 'tit': 'Rude girl', 'au...","[{'sig': 'XI A 5161'}, {'sig': '2022 A 11177'}]"
1,2,9783868215342,"[{'idn': '1049849752', 'tit': 'Bilderbuch und ...","[{'sig': 'Ga 2994'}, {'sig': '2014 A 59299'}]"
2,3,9783779965275,"[{'idn': '1231247207', 'tit': 'Kritische sozia...","[{'sig': '2022 A 9903'}, {'sig': '2022 AA 1982..."
3,4,9783826076473,"[{'idn': '1257538810', 'tit': 'Was ist Schönh...","[{'sig': '2022 A 31731'}, {'sig': '2023 A 5433'}]"
4,5,9783518299197,"[{'idn': '1197070176', 'tit': 'Down girl', 'au...","[{'sig': '2020 A 65123'}, {'sig': '2020 A 5844..."
5,6,9783593509938,"[{'idn': '1162456698', 'tit': 'Wie männlich i...","[{'sig': '2018 A 90189'}, {'sig': '2020 A 1837..."
6,7,9783960420545,"[{'idn': '1172148945', 'tit': 'Politiken der I...","[{'sig': '2019 A 53309'}, {'sig': '2019 A 3146..."
7,8,9783960420552,"[{'idn': '1172149658', 'tit': 'Subjekte der In...","[{'sig': '2019 A 57599'}, {'sig': '2019 A 3112..."
8,9,9783960420569,"[{'idn': '1172148759', 'tit': 'Theorien der In...","[{'sig': '2019 A 57601'}, {'sig': '2019 A 3112..."
9,10,3937390227,"[{'idn': '971153302', 'tit': 'Strategien der W...","[{'sig': 'Ra 147'}, {'sig': '2004 B 10702'}]"


In [50]:
df.to_excel('isbntestb.xlsx')